# Media Pagination

> Factory function for creating paginated media browser views

In [ ]:
#| default_exp media.pagination

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Callable
from fasthtml.common import *

from cjm_fasthtml_interactions.patterns.pagination import Pagination, PaginationStyle

from cjm_fasthtml_workflow_transcription_single_file.media.library import MediaLibrary
from cjm_fasthtml_workflow_transcription_single_file.media.mounter import MediaMounter
from cjm_fasthtml_workflow_transcription_single_file.media.scanner import MediaScanner
from cjm_fasthtml_workflow_transcription_single_file.media.components import (
    grid_view_content, list_view_content, media_browser_controls, empty_media_content
)

## create_media_pagination

Factory function that creates a `Pagination` instance configured for browsing media files with grid/list views and filtering.

In [ ]:
#| export
def create_media_pagination(
    pagination_id: str,
    scanner: MediaScanner,
    mounter: MediaMounter,
    items_per_page: int = 30,
    content_id: Optional[str] = None,
    preview_route_func=None,
    modal_id: str = "sf-media-preview"
) -> Pagination:
    """Create a Pagination instance for media browsing.

    Args:
        pagination_id: Unique identifier for this pagination instance.
        scanner: MediaScanner instance for loading files.
        mounter: MediaMounter instance for URL generation.
        items_per_page: Number of items per page.
        content_id: HTML ID for content area.
        preview_route_func: Function to generate preview route URL.
        modal_id: ID for the preview modal.

    Returns:
        Configured Pagination instance.
    """

    def load_media_files(request) -> List[MediaFile]:
        """Load media files from scanner with optional filtering."""
        files = scanner.scan()

        # Filter by media type if specified
        media_type = request.query_params.get("media_type")
        if media_type and media_type != "all":
            files = [f for f in files if f.media_type == media_type]

        return files

    def render_media_items(items: List[MediaFile], page: int, request) -> Any:
        """Render media items for the current page."""
        view_mode = request.query_params.get("view", "grid")
        media_type = request.query_params.get("media_type")
        start_idx = (page - 1) * items_per_page

        if items:
            if view_mode == "grid":
                return grid_view_content(
                    media_files=items,
                    mounter=mounter,
                    start_idx=start_idx,
                    media_type=media_type,
                    preview_route_func=preview_route_func,
                    modal_id=modal_id
                )
            else:
                return list_view_content(
                    media_files=items,
                    mounter=mounter,
                    start_idx=start_idx,
                    media_type=media_type,
                    preview_route_func=preview_route_func,
                    modal_id=modal_id
                )
        else:
            return empty_media_content(
                message="No media files found. Configure directories in settings.",
            )

    return Pagination(
        pagination_id=pagination_id,
        data_loader=load_media_files,
        render_items=render_media_items,
        items_per_page=items_per_page,
        content_id=content_id,
        preserve_params=["view", "media_type"],
        show_endpoints=True,
        push_url=False  # Don't push URL for workflow-internal browsing
    )

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()